In [1]:
import torch
import torchvision
import cv2 as cv
import time
import traitlets
import ipywidgets.widgets as widgets
import numpy as np
from IPython.display import display
from jetbot import bgr8_to_jpeg
from jetbot.camera.zmq_camera import ZmqCamera as Camera
from stable_baselines3.common.policies import *
from stable_baselines3 import PPO
import gym
from gym import spaces
from jetbot_model import CustomCNN
from jetbot import Robot


In [2]:
#MODEL_PATH = 'best_model_loc_smaller_penalty.zip'
MODEL_PATH = 'final_grayscale_best_model.zip'
model = PPO.load(MODEL_PATH, device='cpu')
device = torch.device('cuda')
model.device = device
model.rollout_buffer.device = device
model.policy = model.policy.to(device)
model.policy.device = device


/usr/local/lib/python3.6/dist-packages/cloudpickle/cloudpickle.py:704: UserWarning: A pickle file created using an old (<=1.4.1) version of cloudpicke is currently being loaded. This is not supported by cloudpickle and will break in cloudpickle 1.7
  "will break in cloudpickle 1.7", category=UserWarning


In [3]:
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
    
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

turning_slider = widgets.FloatSlider(description='turning', min=-2.0, max=2.0, orientation='horizontal')

display(widgets.VBox([image, turning_slider]))

In [4]:
robot = Robot()
state = np.zeros((224,224,1))
gain = 0.155
const_gas = 0.008
while True:
   
    state = cv.cvtColor(camera.value, cv.COLOR_BGR2GRAY)
    state = state.reshape((state.shape[0], state.shape[1], 1))

    action = model.predict(state)
    action = np.clip(action[0],0,2)
    left, right = float(action[0]), float(action[1])
    
    turning_slider.value = left - right
    
    robot.set_motors(const_gas + gain * left, const_gas + gain * right)

    time.sleep(0.001)
    

KeyboardInterrupt: 

In [5]:
robot.stop()
